In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Arrhythmia"
cohort = "GSE235307"

# Input paths
in_trait_dir = "../../input/GEO/Arrhythmia"
in_cohort_dir = "../../input/GEO/Arrhythmia/GSE235307"

# Output paths
out_data_file = "../../output/preprocess/Arrhythmia/GSE235307.csv"
out_gene_data_file = "../../output/preprocess/Arrhythmia/gene_data/GSE235307.csv"
out_clinical_data_file = "../../output/preprocess/Arrhythmia/clinical_data/GSE235307.csv"
json_path = "../../output/preprocess/Arrhythmia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression and atrial fibrillation prediction"
!Series_summary	"The aim of this study was to identify a blood gene expression profile that predicts atrial fibrillation in heart failure patients"
!Series_overall_design	"Cardiac blood samples were obtained from the coronary sinus during CRT-D (Cardiac Resynchronization Therapy - Defibrillator) placement in heart failure patients. Patients were followed during 1 year."
Sample Characteristics Dictionary:
{0: ['tissue: Whole blood'], 1: ['gender: Male', 'gender: Female'], 2: ['age: 63', 'age: 60', 'age: 72', 'age: 66', 'age: 70', 'age: 64', 'age: 61', 'age: 44', 'age: 54', 'age: 50', 'age: 79', 'age: 51', 'age: 55', 'age: 67', 'age: 52', 'age: 73', 'age: 76', 'age: 43', 'age: 68', 'age: 78', 'age: 69', 'age: 57', 'age: 59', 'age: 53', 'age: 65', 'age: 56', 'age: 74', 'age: 38', 'age: 71', 'age: 37'], 3: ['cardiopathy: ischemic', 'cardiopathy: non ischemic', 'cardiopathy: mixed'], 4: ['cardiac rhyt

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np

# 1. Gene Expression Data Availability
is_gene_available = True  # Based on Series info, this seems to be a gene expression dataset

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 5  # "cardiac rhythm after 1 year follow-up" indicates Atrial fibrillation presence
age_row = 2    # Age information is available in row 2
gender_row = 1  # Gender information is available in row 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary: 1 for atrial fibrillation, 0 for sinus rhythm."""
    if value is None or not isinstance(value, str):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'atrial fibrillation' in value:
        return 1
    elif 'sinus rhythm' in value:
        return 0
    return None

def convert_age(value):
    """Convert age value to a continuous numeric value."""
    if value is None or not isinstance(value, str):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    else:
        value = value.strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary: 1 for male, 0 for female."""
    if value is None or not isinstance(value, str):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'male' in value:
        return 1
    elif 'female' in value:
        return 0
    return None

# 3. Save Metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Ensure the directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Assuming clinical_df is available from a previous step
    # If not available, we would need to load it first
    try:
        # Extract clinical features using the correct parameter name
        clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Use the existing clinical_data variable
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(clinical_df)
        print("Clinical Data Preview:", preview)
        
        # Save to CSV
        clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Error: clinical_data is not defined. Make sure to load the clinical data first.")


Clinical Data Preview: {'GSM7498589': [0.0, 63.0, 1.0], 'GSM7498590': [0.0, 60.0, 1.0], 'GSM7498591': [0.0, 60.0, 1.0], 'GSM7498592': [0.0, 72.0, 1.0], 'GSM7498593': [0.0, 63.0, 1.0], 'GSM7498594': [0.0, 66.0, 1.0], 'GSM7498595': [0.0, 70.0, 1.0], 'GSM7498596': [0.0, 64.0, 1.0], 'GSM7498597': [0.0, 63.0, 1.0], 'GSM7498598': [0.0, 61.0, 1.0], 'GSM7498599': [0.0, 70.0, 1.0], 'GSM7498600': [0.0, 64.0, 1.0], 'GSM7498601': [0.0, 63.0, 1.0], 'GSM7498602': [0.0, 44.0, 1.0], 'GSM7498603': [0.0, 54.0, 1.0], 'GSM7498604': [0.0, 44.0, 1.0], 'GSM7498605': [0.0, 50.0, 1.0], 'GSM7498606': [1.0, 79.0, 1.0], 'GSM7498607': [0.0, 63.0, 1.0], 'GSM7498608': [0.0, 63.0, 1.0], 'GSM7498609': [1.0, 64.0, 1.0], 'GSM7498610': [0.0, 60.0, 1.0], 'GSM7498611': [0.0, 51.0, 1.0], 'GSM7498612': [0.0, 55.0, 1.0], 'GSM7498613': [0.0, 55.0, 1.0], 'GSM7498614': [1.0, 67.0, 1.0], 'GSM7498615': [0.0, 52.0, 1.0], 'GSM7498616': [0.0, 70.0, 1.0], 'GSM7498617': [0.0, 54.0, 1.0], 'GSM7498618': [0.0, 54.0, 1.0], 'GSM7498619': [0

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Arrhythmia/GSE235307/GSE235307_series_matrix.txt.gz


Gene data shape: (58717, 119)
First 20 gene/probe identifiers:
Index(['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers presented in the previous step
# The identifiers appear to be numeric values (4, 5, 6, etc.) rather than standard gene symbols
# Human gene symbols would typically be alphanumeric (like BRCA1, TP53, etc.)
# These numeric identifiers need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'COL', 'ROW', 'NAME', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'LOCUSLINK_ID', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 

Column 'ID' contains values matching gene data ID pattern


Column 'COL' contains values matching gene data ID pattern


Column 'ROW' contains values matching gene data ID pattern


Column 'NAME' contains values matching gene data ID pattern


...

Column 'CHROMOSOMAL_LOCATION' contains values matching gene data ID pattern


Column 'CYTOBAND' contains values matching gene data ID pattern


Column 'DESCRIPTION' contains values matching gene data ID pattern


Column 'GO_ID' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'NAME' may contain gene-related information
Sample values: ['GE_BrightCorner', 'DarkCorner', 'DarkCorner']
Column 'GENE_SYMBOL' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'GENE_NAME' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'UNIGENE_ID' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'DESCRIPTION' may contain gene-related information
Sample values: [nan, nan, nan]


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain the probe IDs and gene symbols
# Based on the previous outputs:
# - 'ID' column in gene_annotation contains numeric identifiers that match gene_data indices
# - 'GENE_SYMBOL' column contains the gene symbols we need to map to

# 2. Get gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Gene mapping preview:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This uses the provided function that handles many-to-many mappings
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
print(f"Gene expression data after mapping: {gene_data.shape}")
print("First few genes and samples:")
print(gene_data.iloc[:5, :3])

# 4. Save the gene data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (54295, 2)
Gene mapping preview:
  ID          Gene
3  4         HEBP1
4  5         KCNE4
5  6        BPIFA3
6  7  LOC100129869
7  8          IRG1


Gene expression data after mapping: (20353, 119)
First few genes and samples:
           GSM7498589   GSM7498590  GSM7498591
Gene                                          
A1BG      1215.921532  1042.240181  860.505266
A1BG-AS1   167.933502   156.514231  153.778492
A1CF       590.313903   592.863373  596.313839
A2LD1      270.359499   377.943542  351.312844
A2M        555.994327   688.515352  651.543646


Gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE235307.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to liver fibrosis progression, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for liver cirrhosis analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (20353, 119)
Gene data shape after normalization: (19847, 119)


Normalized gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE235307.csv


Original clinical data preview:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   
2  !Sample_characteristics_ch1   
3  !Sample_characteristics_ch1   
4  !Sample_characteristics_ch1   

                                          GSM7498589  \
0                                tissue: Whole blood   
1                                       gender: Male   
2                                            age: 63   
3                              cardiopathy: ischemic   
4  cardiac rhythm at start of the study: Sinus rh...   

                                          GSM7498590  \
0                                tissue: Whole blood   
1                                       gender: Male   
2                                            age: 60   
3                              cardiopathy: ischemic   
4  cardiac rhythm at start of the study: Sinus rh...   

                                          GSM7498591  \
0                                

Data shape after handling missing values: (119, 19850)
For the feature 'Arrhythmia', the least common label is '1.0' with 13 occurrences. This represents 10.92% of the dataset.
The distribution of the feature 'Arrhythmia' in this dataset is fine.

Quartiles for 'Age':
  25%: 55.0
  50% (Median): 63.0
  75%: 68.0
Min: 37.0
Max: 79.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 119 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Data shape after removing biased features: (119, 19849)


Linked data saved to ../../output/preprocess/Arrhythmia/GSE235307.csv
